In [1]:
library(shiny)

# UI

In [2]:
ui <- shinyUI(fluidPage(
    titlePanel("Predict horsepower from MPG"),
    sidebarLayout(
        sidebarPanel(
            sliderInput(
                "sliderMPG",
                "What is the MPG of the car?",
                10,
                35,
                value = 20 
            ),
            checkboxInput(
                "showModel1",
                "Show/hide model 1",
                value = TRUE
            ),
            checkboxInput(
                "showModel2",
                "Show/hide model 2",
                value = TRUE
            ),
            submitButton("Submit!") # new
        ),
        mainPanel(
            plotOutput("plot1"),
            h3("Predicted horsepower from model 1:"),
            textOutput("pred1"),
            h3("Predicted horsepower from model 2:"),
            textOutput("pred2")
        )
    )
))

# Server

In [3]:
server <- shinyServer(function(input, output) {
    mtcars$mpgsp <- ifelse(mtcars$mpg - 20 > 0, mtcars$mpg - 20, 0)
    model1 <- lm(hp ~ mpg, data = mtcars)
    model2 <- lm(hp ~ mpgsp + mpg, data = mtcars)
    model1pred <- reactive({
        mpgInput <- input$sliderMPG
        predict(model1, newdata = data.frame(mpg = mpgInput))
    })
    model2pred <- reactive({
        mpgInput <- input$sliderMPG
        predict(
            model2,
            newdata = data.frame(
                mpg = mpgInput,
                mpgsp = ifelse(mpgInput - 20 > 0, mpgInput - 20, 0)
            )
        )
    })
    output$plot1 <- renderPlot({
        mpgInput <- input$sliderMPG
        plot(
            mtcars$mpg,
            mtcars$hp,
            xlab = "Miles per gallon",
            ylab = "Horsepower",
            bty = "n",
            pch = 16,
            xlim = c(10, 35),
            ylim = c(50, 350)
        )
        if(input$showModel1) {
            abline(model1, col = "red", lwd = 2)
        }
        if(input$showModel2) {
            model2line <- predict(
                model2,
                newdata = data.frame(
                    mpg = 10:35,
                    mpgsp = ifelse(10:35 - 20 > 0, 10:35 - 20, 0)
                )
            )
            lines(10:35, model2line, col = "blue", lwd = 2)
        }
        legend(
            25,
            250,
            c("Model 1 prediction", "Model 2 prediction"),
            pch = 16,
            col = c("red", "blue"),
            bty = "n",
            cex = 1.2
        )
        points(mpgInput, model1pred(), col = "red", pch = 16, cex = 2)
        points(mpgInput, model2pred(), col = "blue", pch = 16, cex = 2)
    })
    output$pred1 <- renderText({model1pred()})
    output$pred2 <- renderText({model2pred()})
})

# Run

In [5]:
options(shiny.port = 8080)
runApp(shinyApp(ui, server))


Listening on http://127.0.0.1:8080

